In [1]:
from pprint import pprint

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from utils import *
import fantraxAPI
# from fantraxAPI import *
import baseballprospectus

import tools_cvf

plt.style.use(tools_cvf.get_stylesheets(dark=True))
px.defaults.template = "plotly_dark"


In [2]:
leagueInfo=fantraxAPI.fetch_leagueInfo()

requesting league info. status code: 200


In [3]:
# load player data list

# create a dataframe of player info
df_playerList = pd.DataFrame(
  [[k, *v.values()] for k, v in leagueInfo["playerInfo"].items()],
  columns=["fantraxid", "pos_elig", "status"]
)
# append team details where we can (so far)
df_playerList["team"] = None
df_playerList.loc[df_playerList.status == "FA", "team"] = False
df_playerList.loc[df_playerList.status == "WW", "team"] = False
teamRosters = fantraxAPI.fetch_teamRosters()
for team in teamRosters["rosters"].keys():
  for fantraxid in [ri["id"] for ri in teamRosters["rosters"][team]["rosterItems"]]:
    df_playerList.loc[df_playerList.fantraxid == fantraxid, "team"] = team

# get the team info for each team
df_teamInfo = pd.DataFrame(
  [list(v.values()) for v in leagueInfo["teamInfo"].values()],
  columns=["division", "name", "id"],
)
# add the team info to players now that we have it
df_playerList = df_playerList.merge(df_teamInfo.add_prefix("team_"), how="left", left_on="team", right_on="team_id").drop(columns="team")

# append various player IDs
df_idmap = baseballprospectus.load_playerIDMap()
df_playerList = df_playerList.merge(
  df_idmap[[
    "FANTRAXID", "PLAYERNAME",
    "MLBID", "IDFANGRAPHS", "RETROID", "BREFID", "BPID",
  ]],
  how="left", left_on="fantraxid", right_on="FANTRAXID",
).drop(columns="FANTRAXID").rename(
  columns={
    # "FANTRAXID": "fantraxid",
    "PLAYERNAME": "playername",
    "MLBID": "mlbid",
    "IDFANGRAPHS": "idfangraphs",
    "RETROID": "retroid",
    "BREFID": "brefid",
    "BPID": "bpid",
  }
)
assert np.sum(df_playerList[(df_playerList.status == "T")].mlbid.isnull()) == 0, "seems like active players are missing..."

df_playerList[df_playerList.status == "T"]

requesting team rosters. status code: 200
loading player ID map... done.


,fantraxid,pos_elig,status,team_division,team_name,team_id,playername,mlbid,idfangraphs,retroid,brefid,bpid
178,04b02,"OF,UT",T,AOA West,Salt Lake Fireflies,i1xlfdkplrs9xvi3,Seiya Suzuki,673548,30116,NaN,suzukse01,138164
199,04azv,"3B,UT",T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Jake Burger,669394,22275,NaN,burgeja01,109519
219,04auj,SP,T,AOA East,Amontillado Trowels,1770mj2elrsgambq,Framber Valdez,664285,17295,valdf001,valdefr01,107058
305,029si,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Eduardo Rodriguez,593958,13164,rodre102,rodried05,67588
360,04ai4,"2B,UT",T,AOA West,Salt Lake Fireflies,i1xlfdkplrs9xvi3,Nolan Gorman,669357,22263,NaN,gormano01,121931
...,...,...,...,...,...,...,...,...,...,...,...,...
14764,000dp,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Justin Verlander,434378,8700,verlj001,verlaju01,45613
15137,05aoc,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Eury Perez,691587,27768,NaN,perezeu02,148223
15148,05ajg,"2B,UT,3B,SS",T,AOA West,Airsick Lowlanders,nrrvbec0lrsgambq,Ha-Seong Kim,673490,27506,NaN,kimha01,125523
15248,04a2t,SP,T,AOA East,District Dingers,5om1y9lglrsgambq,Kodai Senga,673540,31838,NaN,sengako01,136521


In [4]:
df_hitters, df_pitchers = baseballprospectus.load_baseballprospectus_data()
df_hitters = baseballprospectus.append_fantraxIDs(df_hitters, df_idmap=df_idmap)
df_pitchers = baseballprospectus.append_fantraxIDs(df_pitchers, df_idmap=df_idmap)
df_hitters = baseballprospectus.append_fantrax_scoring(df_hitters, pitching=False)
df_pitchers = baseballprospectus.append_fantrax_scoring(df_pitchers, pitching=True)
df_bp_merged = baseballprospectus.merge_hittingpitching(df_hitters, df_pitchers)

loading PECOTA hitting... done.
loading PECOTA pitching... done.


In [5]:
df_playerList_bp = df_playerList.merge(df_bp_merged, how="left", left_on="fantraxid", right_on="fantraxid", suffixes=(None, "_bp"))
df_playerList_bp.loc[df_playerList_bp.team_id.isna(), "team_name"] = "FA"
df_playerList_bp

,fantraxid,pos_elig,status,team_division,team_name,team_id,playername,mlbid,idfangraphs,retroid,...,fip,cfip,dra,dra_minus,warp_pitch,comparables_pitch,pos_pitch,fpts_pitch,warp,fpts
0,01yqi,SP,WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028jh,"SS,UT",WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028is,SP,FA,NaN,FA,NaN,Vincent Velasquez,592826,11189,velav001,...,5.01,119.0,5.27,113.0,-0.1,"Stan Bahnsen (65), Sterling Hitchcock (65), Sc...",P,59.492868,-0.1,59.492868
3,028it,RP,FA,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06fz8,"OF,UT",WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15481,06er5,SP,WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15482,06er6,"C,UT",FA,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15483,06er4,"C,UT",WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15484,06g2a,SP,WW,NaN,FA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_playerList_bp[df_playerList_bp.status == "T"]

,fantraxid,pos_elig,status,team_division,team_name,team_id,playername,mlbid,idfangraphs,retroid,...,fip,cfip,dra,dra_minus,warp_pitch,comparables_pitch,pos_pitch,fpts_pitch,warp,fpts
178,04b02,"OF,UT",T,AOA West,Salt Lake Fireflies,i1xlfdkplrs9xvi3,Seiya Suzuki,673548,30116,NaN,...,0.00,0.0,0.00,0.0,0.0,NaN,NaN,0.000000,3.0,161.750000
199,04azv,"3B,UT",T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Jake Burger,669394,22275,NaN,...,0.00,0.0,0.00,0.0,0.0,NaN,NaN,0.000000,1.3,110.500000
219,04auj,SP,T,AOA East,Amontillado Trowels,1770mj2elrsgambq,Framber Valdez,664285,17295,valdf001,...,3.71,76.0,3.90,83.0,3.5,"CC Sabathia (77), Bob Gibson (75), Denny Neagl...",SP,303.131923,3.5,303.131923
305,029si,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Eduardo Rodriguez,593958,13164,rodre102,...,4.34,99.0,4.78,102.0,1.2,"Jon Lester (81), Cole Hamels (78), Tom Glavine...",SP,207.083216,1.2,207.083216
360,04ai4,"2B,UT",T,AOA West,Salt Lake Fireflies,i1xlfdkplrs9xvi3,Nolan Gorman,669357,22263,NaN,...,0.00,0.0,0.00,0.0,0.0,NaN,NaN,0.000000,1.2,92.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14764,000dp,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Justin Verlander,434378,8700,verlj001,...,4.46,102.0,4.48,96.0,1.6,"Roger Clemens (79), Don Sutton (78), Tom Seave...",SP,203.499639,1.6,203.499639
15137,05aoc,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq,Eury Perez,691587,27768,NaN,...,4.05,91.0,4.26,91.0,1.7,"Tyler Skaggs (67), Deivi García (65), Taijuan ...",SP,184.196848,1.7,184.196848
15148,05ajg,"2B,UT,3B,SS",T,AOA West,Airsick Lowlanders,nrrvbec0lrsgambq,Ha-Seong Kim,673490,27506,NaN,...,0.00,0.0,0.00,0.0,0.0,NaN,NaN,0.000000,2.7,136.250000
15248,04a2t,SP,T,AOA East,District Dingers,5om1y9lglrsgambq,Kodai Senga,673540,31838,NaN,...,4.24,95.0,4.24,90.0,1.2,"Dave Stewart (90), David Cone (90), Bob Veale ...",SP,130.543065,1.2,130.543065


In [7]:
fig = px.scatter(
  df_playerList_bp[["fantraxid", "mlbid", "pos_elig", "name", "warp", "fpts", "team_name"]],
  x="warp",
  y="fpts",
  color="team_name",
  hover_data=["name", "pos_elig", "fantraxid", "mlbid"],
  color_discrete_map={
    "FA": "gray",
  },
)
fig.show()

In [8]:
import pybaseball

data_pyb_batting = pybaseball.batting_stats(2024).rename(columns={
  "PA": "pa",
  "R": "r",
  "1B": "b1",
  "2B": "b2",
  "3B": "b3",
  "HR": "hr",
  "RBI": "rbi",
  "BB": "bb",
  "HBP": "hbp",
  "SO": "so",
  "SB": "sb",
  "CS": "cs",
  "GDP": "gidp",
  "IBB": "ibb",
  "SF": "sf",
  "SH": "sh",
})
data_pyb_pitching = pybaseball.pitching_stats(2024).rename(columns={
  "W": "w",
  "L": "l",
  "SV": "sv",
  "HLD": "hld",
  # "QS": "qs",
  "IP": "ip",
  "HR": "hr",
  "BB": "bb",
  "HBP": "hbp",
  "ER": "er",
  "SO": "so",
  "BK": "bk",
  "BS": "bs",
  # "IR": "ir",
  # "IRS": "irs",
  "IBB": "ibb",
  "WP": "wp",
})
data_pyb_batting_br = pybaseball.batting_stats_bref(2024).rename({
  "PO": "pko",
})
data_pyb_pitching_br = pybaseball.pitching_stats_bref(2024).rename({
  "1B": "1b",
  "2B": "2b",
  "3B": "3b",
})

data_pyb_batting["IDfg"] = data_pyb_batting.IDfg.astype(str)
data_pyb_pitching["IDfg"] = data_pyb_pitching.IDfg.astype(str)

In [9]:
# for i, k in enumerate(data_pyb_pitching.keys()):
#   print(k, end=", " if (i+1) % 8 != 0 else None)

In [19]:
for i, k in enumerate(data_pyb_pitching_br.keys()):
  print(k, end=", " if (i+1) % 8 != 0 else None)

Name, Age, #days, Lev, Tm, G, GS, W
L, SV, IP, H, R, ER, BB, SO
HR, HBP, ERA, AB, 2B, 3B, IBB, GDP
SF, SB, CS, PO, BF, Pit, Str, StL
StS, GB/FB, LD, PU, WHIP, BAbip, SO9, SO/W
mlbID, 

In [25]:
df_pyb_pitching_all = data_pyb_pitching.merge(
  df_idmap,
  how="left",
  left_on="IDfg",
  right_on="IDFANGRAPHS",
).merge(
  data_pyb_pitching_br,
  how="left",
  left_on="MLBID",
  right_on="mlbID",
  suffixes=[None,"_BR"],
)
# df_pyb_pitching_all[df_pyb_pitching_all.WHIP_BR.isna()]
df_pyb_pitching_all.columns

,IDfg,Season,Name,Team,Age,w,l,WAR,ERA,G,...,StL,StS,GB/FB_BR,LD_BR,PU,WHIP_BR,BAbip,SO9,SO/W,mlbID
8,27863,2024,Jared Jones,PIT,22,1,0,0.3,4.76,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,19924,2024,Bryan Hudson,MIL,27,0,0,0.2,0.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,24614,2024,Victor Vodnik,COL,24,0,0,0.0,0.00,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,25645,2024,Declan Cronin,MIA,26,0,1,0.0,0.00,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_idmap[df_idmap["FANTRAXNAME"].str.contains("Hudson")==True]

,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,...,OTTONEUID,HQID,RAZZBALLNAME,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE,UNDERDOG
1387,hudsoda01,Daniel Hudson,3/9/1987,Daniel,Hudson,LAD,NL,P,7146,Daniel Hudson,...,9304.0,3340.0,Daniel Hudson,01fd0,Daniel Hudson,Daniel Hudson,P,"Hudson, Daniel",Y,dd9f1cba-f016-48f1-b014-9d15bea6dc0d
1388,hudsoda02,Dakota Hudson,9/15/1994,Dakota,Hudson,COL,NL,P,19206,Dakota Hudson,...,23707.0,5913.0,Dakota Hudson,041md,Dakota Hudson,Dakota Hudson,SP/RP,"Hudson, Dakota",Y,867f65ce-842a-440a-b980-2f1f1b41f6b8
1389,hudsoor01,Orlando Hudson,12/12/1977,Orlando,Hudson,NaN,NaN,2B,1307,Orlando Hudson,...,15196.0,NaN,Orlando Hudson,000k5,Orlando Hudson,Orlando Hudson,2B,"Hudson, Orlando",N,NaN
1390,hudsoti01,Tim Hudson,7/14/1975,Tim,Hudson,NaN,NaN,P,921,Tim Hudson,...,12427.0,NaN,Tim Hudson,nan,Tim Hudson,Tim Hudson,P,"Hudson, Tim",N,NaN
